In [1]:
import numpy as np

import scipy.sparse        as sparse
import scipy.sparse.linalg as lin

import utilitis_FEEC.mass_matrices         as mass
import utilitis_FEEC.bsplines              as bsp
import utilitis_FEEC.mass_matrices_stencil as mass_sten


import psydac as ps


from psydac.linalg.iterative_solvers import cg
from psydac.fem.basic                import FemField
from psydac.linalg.stencil           import StencilMatrix, StencilVector, StencilVectorSpace
from psydac.linalg.block             import ProductSpace, BlockVector, BlockLinearOperator, BlockMatrix

In [2]:
import utilitis_FEEC.mappings    as mps
import utilitis_FEEC.derivatives as der
import utilitis_FEEC.evaluation  as eva

In [3]:
p       = [4, 3, 3]
bc      = [False, True, True]
Nel     = [8, 7, 6]
el_b    = [np.linspace(0., 1., Nel + 1) for Nel in Nel]
T       = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]
Nbase   = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]

#mapping = ['torus', 2., 0.2, 1.]
mapping = ['hollow cylinder', 0.2, 1., 1.]
#mapping = ['slab', 1., 2., 3.]

mapp = mps.mappings(mapping)
deri = der.discrete_derivatives(p, T, bc)

In [25]:
# Create 1D finite element spaces
V1 = ps.fem.splines.SplineSpace(p[0], grid=el_b[0], periodic=bc[0])
V2 = ps.fem.splines.SplineSpace(p[1], grid=el_b[1], periodic=bc[1])
V3 = ps.fem.splines.SplineSpace(p[2], grid=el_b[2], periodic=bc[2])

# Create 3D tensor product finite element space
V = ps.fem.tensor.TensorFemSpace(V1, V2, V3)

In [26]:
M0 = mass.mass_V0(T, p, bc, mapp.g_sqrt)

In [27]:
M0

<504x504 sparse matrix of type '<class 'numpy.float64'>'
	with 155232 stored elements in Compressed Sparse Row format>

In [28]:
M0_sten = mass_sten.mass_matrix_V0(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, mapp.g_sqrt, bc)

In [29]:
np.allclose(M0_sten.toarray(), M0.toarray())

True

In [30]:
fun = lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi)

In [32]:
F0 = mass.inner_prod_V0(T, p, bc, mapp.g_sqrt, fun)

In [33]:
F0.shape

(12, 7, 6)

In [60]:
F0_sten = mass_sten.inner_prod_V0(V, fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [61]:
np.allclose(F0.flatten(), F0_sten.toarray())

True

In [62]:
coeff, info = lin.cg(M0, F0.flatten(), tol=1e-10) 

In [63]:
coeff_sten, info = cg(M0_sten, F0_sten, tol=1e-10)

In [64]:
np.allclose(coeff.flatten(), coeff_sten.toarray())

True

In [66]:
coeff = np.reshape(coeff, (12, 7, 6))

In [67]:
error = ma.L2_error_V0(coeff, T, p, bc, g_sqrt, fun)

In [68]:
error

0.04381653304415181

In [69]:
# ... create Fem field
fun_h = FemField(V, coeffs=coeff_sten)
fun_h.coeffs.update_ghost_regions()

integrand = lambda *x : (fun(*x) - fun_h(*x))**2*g_sqrt(*x) 

In [70]:
error_sten = np.sqrt(V.integral(integrand))

In [71]:
error_sten

0.043816533044151465

In [72]:
M3 = ma.mass_V3(T, p, bc, g_sqrt)

In [73]:
M3

<462x462 sparse matrix of type '<class 'numpy.float64'>'
	with 68250 stored elements in Compressed Sparse Row format>

In [74]:
M3_sten = ma_sten.mass_matrix_V3(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [75]:
np.allclose(M3_sten.toarray(), M3.toarray())

True

In [76]:
M1 = ma.mass_V1(T, p, bc, Ginv, g_sqrt)

In [77]:
M1_sten, V1 = ma_sten.mass_matrix_V1(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, Ginv, g_sqrt, bc)

In [78]:
from psydac.linalg.block import BlockMatrix

M1_sten = BlockMatrix(V1, V1, blocks=M1_sten).tosparse()

In [79]:
np.allclose(M1_sten.toarray(), M1.toarray())

True

In [80]:
M2 = ma.mass_V2(T, p, bc, G, g_sqrt)

In [81]:
M2_sten, V2 = ma_sten.mass_matrix_V2(p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, G, g_sqrt, bc)

In [82]:
from psydac.linalg.block import BlockMatrix

M2_sten = BlockMatrix(V2, V2, blocks=M2_sten).tosparse()

In [83]:
np.allclose(M2_sten.toarray(), M2.toarray())

True

In [84]:
fun = [lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi), lambda r, phi, z : r**2*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi), lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi), lambda r, phi, z : r*np.sin(2*np.pi*r)*np.cos(4*np.pi*phi)]

In [85]:
F1 = ma.inner_prod_V1(T, p, bc, Ginv, g_sqrt, fun)

In [86]:
F1 = np.concatenate((F1[0].flatten(), F1[1].flatten(), F1[2].flatten()))

In [87]:
F1_sten, V1 = ma_sten.inner_prod_V1(fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, Ginv, g_sqrt, bc)

In [88]:
F1_sten = np.concatenate((F1_sten[0].toarray(), F1_sten[1].toarray(), F1_sten[2].toarray()))

In [89]:
np.allclose(F1, F1_sten)

True

In [90]:
F2 = ma.inner_prod_V2(T, p, bc, G, g_sqrt, fun)

In [91]:
F2 = np.concatenate((F2[0].flatten(), F2[1].flatten(), F2[2].flatten()))

In [92]:
F2.shape

(1428,)

In [93]:
F2_sten, V2 = ma_sten.inner_prod_V2(fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, G, g_sqrt, bc)

In [94]:
F2_sten = np.concatenate((F2_sten[0].toarray(), F2_sten[1].toarray(), F2_sten[2].toarray()))

In [95]:
np.allclose(F2, F2_sten)

True

In [96]:
fun = lambda r, phi, z : r*np.sin(2*np.pi*r)*np.sin(4*np.pi*phi)

In [97]:
F3 = ma.inner_prod_V3(T, p, bc, g_sqrt, fun)

In [98]:
F3.shape

(11, 7, 6)

In [99]:
F3_sten = ma_sten.inner_prod_V3(fun, p, [Nel[0] + p[0], Nel[1] + p[1], Nel[2] + p[2]], T, g_sqrt, bc)

In [100]:
np.allclose(F3.flatten(), F3_sten.toarray())

True